# **Sound Spot**

##### Installation et importations

In [1]:
# %pip install spotipy
# %pip install selenium
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
import json

import requests
import time

from selenium import webdriver

Authorization Site : https://accounts.spotify.com/fr/authorize?client_id=ee862f2c84fe4f00929c71fb2eb50807&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com&scope=playlist-read-private

In [ ]:
# TODO: VARIABLES D'ENVIRONNEMENT

In [2]:
# IDs
client_id = input('CLIENT_ID : ')
client_secret = input('CLIENT_SECRET : ')

##### Accéder aux infos Spotify de l'utilisateurs pour créer notre BDD

In [3]:
# Créez une instance de Spotipy avec des identifiants client
sp_oauth = SpotifyOAuth(
    client_id = client_id,
    client_secret = client_secret,
    redirect_uri = 'https://open.spotify.com',
    scope = 'user-library-read'  # Autorisations nécessaires, y compris la lecture des playlists privées
)


In [4]:
# Obtenir le jeton d'accès ou le renouveler s'il est expiré
token_info = sp_oauth.get_cached_token()
print("TOKEN EXISTS : ", token_info)

# Vérifiez si le jeton d'accès est valide
if not token_info:

    auth_url = sp_oauth.get_authorize_url()

    driver = webdriver.Chrome()

    driver.get(auth_url)

    # Attendez que l'utilisateur autorise l'application et soit redirigé
    input("Appuyez sur Entrée une fois redirigé vers l'URL de redirection...")

    final_url = driver.current_url

    driver.quit()
    
    response = requests.get(final_url)
    print('RESPONSE : ', response.url)
    code = sp_oauth.get_authorization_code(response.url)
    print("CODE : ", code)

    access_token = sp_oauth.get_access_token(code, as_dict = False)
    print("TOKEN : ", access_token)

else:

    access_token = token_info['access_token']

TOKEN EXISTS :  None
RESPONSE :  https://open.spotify.com/?code=AQBmpP84UGjeP8ee_Mf05zApTMOgCuoTf-OrSZlM5VmtEqyreHZm2gos1Gyw_BNRfoemKG1Nqy-W_xgJyXaI9wqYNzMUTPDxcw5FAH6bQQ0ZFeOwaBDTz4OAIsJsiY-vifIG23sKUioer6pT0qFri59HEvv3bInUtXvqBTdlRB1L47u_zBlv0vN8KfLqpr4
CODE :  AQBmpP84UGjeP8ee_Mf05zApTMOgCuoTf-OrSZlM5VmtEqyreHZm2gos1Gyw_BNRfoemKG1Nqy-W_xgJyXaI9wqYNzMUTPDxcw5FAH6bQQ0ZFeOwaBDTz4OAIsJsiY-vifIG23sKUioer6pT0qFri59HEvv3bInUtXvqBTdlRB1L47u_zBlv0vN8KfLqpr4
TOKEN :  BQAg_-bG_aNostWrVJE6YawbiHr_j9ymAsk0564TYT_na2CPyIa72NVOPFyCwdYFHwhK1r-vEQXJKSPLTNfDyjfQ6_67wTpByKGb7Eht6R1ach-5tKK-aaLV6HPROuzN3J_1UCJ2QhzEAoWDeSNFh_38Ov6SaPSgg2_Mxq7N6dZPGt_rVU-HxLwjlWU_q7aGM-8h1VyYtZ2PG9A


In [5]:
# S'authentifier avec le jeton d'accès
sp = spotipy.Spotify(auth=access_token)
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# ID de la playlist Spotify que vous souhaitez récupérer (vous pouvez obtenir l'ID depuis l'URL de la playlist)
playlist_id = 'https://open.spotify.com/playlist/4QnaHIvkm489VZOp4CWeTB?si=e11e9a34315143db&pt=3c156a8c357198ee10d0f85f529b02b3' # Private Playlist
# playlist_id = 'https://open.spotify.com/playlist/37i9dQZEVXbhGLvgw0jIZm?si=d1e5dfd1966c45b8' # Public Playlist

# Récupérez les informations sur la playlist
playlist = sp.playlist(playlist_id)

# Imprimez les informations sur la playlist
print(f"Nom de la playlist : {playlist['name']}")
print(f"Propriétaire de la playlist : {playlist['owner']['display_name']}")
print(f"Nombre de pistes dans la playlist : {playlist['tracks']['total']}")

# Liste des noms des pistes (en paginant)
track_data = []
results = sp.playlist_tracks(playlist_id)
data = pd.DataFrame()

while results:
    for track in results['items']:
        # Track info
        track_info = track['track']

        # Artists info
        # TODO : Trouver le classement mondial du main artist
        artists = track['track']['artists']
        other_artists = []

        main_artist_id = track_info['artists'][0]['id']
        main_artist_info = sp.artist(main_artist_id)

        for artist in artists:
            other_artists.append(artist['name'])
        
        other_artists.pop(0)

        if 'images' in main_artist_info and main_artist_info['images']:
            main_artist_image_url = main_artist_info['images'][0]['url']
        elif 'image' in main_artist_info:
            main_artist_image_url = main_artist_info['image']['url']
        else:
            main_artist_image_url = None

        track_data.append([
            track_info['id'],
            main_artist_id,
            track_info['name'],
            track_info['popularity'],
            track_info['album']['release_date'],
            main_artist_info['name'],
            main_artist_info['genres'],
            main_artist_info['popularity'],
            other_artists,
            track_info['external_urls'],
            track_info['album']['images'][0]['url'],
            main_artist_image_url
        ])
        
    results = sp.next(results)  # Chargez la page suivante

for idx, song_info in enumerate(track_data):

    data = data.append(
        {
            'Track ID' : song_info[0],
            'Main Artist ID' : song_info[1],
            'Track Name': song_info[2],
            'Track Popularity': song_info[3],
            'Track Released Date' : song_info[4],
            'Main Artist': song_info[5],
            'Main Artist Genre': song_info[6],
            'Main Artist Popularity': song_info[7],
            'Other Artists': song_info[8],
            'Track URL' : song_info[9],
            'Track Image URL' : song_info[10],
            'Main Artist Image URL' : song_info[11]
        },
        ignore_index=True
    )

Nom de la playlist : OK Mood
Propriétaire de la playlist : Chelsie
Nombre de pistes dans la playlist : 166


C:\Users\louis\AppData\Local\Temp\ipykernel_9680\127091064.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(
C:\Users\louis\AppData\Local\Temp\ipykernel_9680\127091064.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(
C:\Users\louis\AppData\Local\Temp\ipykernel_9680\127091064.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(
C:\Users\louis\AppData\Local\Temp\ipykernel_9680\127091064.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(
C:\Users\louis\AppData\Local\Temp\ipykernel_9680\127091064.py:66: FutureWarning: The frame.append method is deprecat

In [6]:
data.head(5)

,Track ID,Main Artist ID,Track Name,Track Popularity,Track Released Date,Main Artist,Main Artist Genre,Main Artist Popularity,Other Artists,Track URL,Track Image URL,Main Artist Image URL
0,2UN0lp72LAusrXi8LLVomt,6eUKZXaKkcviH0Ku9w2n3V,2step (feat. Lil Baby),67,2022-04-22,Ed Sheeran,"[pop, singer-songwriter pop, uk pop]",88,[Lil Baby],{'spotify': 'https://open.spotify.com/track/2U...,https://i.scdn.co/image/ab67616d0000b273fa1f47...,https://i.scdn.co/image/ab6761610000e5eb3bcef8...
1,357EwLZfD9vuGTZ7dxqumw,7wlFDEWiM5OoIAt8RSli8b,Akbar,56,2018-06-15,YoungBoy Never Broke Again,"[baton rouge rap, rap]",82,[],{'spotify': 'https://open.spotify.com/track/35...,https://i.scdn.co/image/ab67616d0000b273b233d2...,https://i.scdn.co/image/ab6761610000e5eba213bd...
2,32vE1nuG8T9c8bhmZdRY6d,246dkjvS1zLTtiykXe5h60,Cooped Up (with Roddy Ricch),73,2022-06-03,Post Malone,"[dfw rap, melodic rap, pop, rap]",88,[Roddy Ricch],{'spotify': 'https://open.spotify.com/track/32...,https://i.scdn.co/image/ab67616d0000b273343626...,https://i.scdn.co/image/ab6761610000e5eb6be070...
3,4ZpIuzx91EAPK3VimONbfB,1rpxM2mBJnWdfPeieLDYOc,FADE UP,70,2022-06-24,ZEG P,[],50,"[Hamza, SCH]",{'spotify': 'https://open.spotify.com/track/4Z...,https://i.scdn.co/image/ab67616d0000b2730568d3...,https://i.scdn.co/image/ab6761610000e5eb0199da...
4,27NovPIUIRrOZoCHxABJwK,7jVv8c5Fj3E9VhNjxT4snq,INDUSTRY BABY (feat. Jack Harlow),81,2021-07-23,Lil Nas X,[lgbtq+ hip hop],76,[Jack Harlow],{'spotify': 'https://open.spotify.com/track/27...,https://i.scdn.co/image/ab67616d0000b273ba2667...,https://i.scdn.co/image/ab6761610000e5ebd66f1e...


##### **Retrouver les infos d'une chanson juste en tapant le nom de la chanson et celui de l'artiste**

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configurez l'authentification
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Remplacez ces valeurs par le nom de la chanson et de l'artiste que vous recherchez
chanson = 'The Hills'
artiste = 'The Weeknd'

# Recherchez la chanson
results = sp.search(q=f'track:{chanson} artist:{artiste}', type='track')

# Obtenez les données de la première chanson de la liste de résultats
if len(results['tracks']['items']) > 0:
    chanson_data = results['tracks']['items'][0]
    print('Nom de la chanson:', chanson_data['name'])
    print('Artiste:', chanson_data['artists'][0]['name'])
    print('Album:', chanson_data['album']['name'])
    print('URL Spotify:', chanson_data['external_urls']['spotify'])
else:
    print('Chanson non trouvée.')


Nom de la chanson: The Hills
Artiste: The Weeknd
Album: The Highlights
URL Spotify: https://open.spotify.com/track/33kxP9sQOzB3fRL8zl17uK


##### MySQL Connection

In [3]:
import mysql.connector

# Create a connexion to the database
connexion = mysql.connector.connect(
    host='localhost',
    user='soundspothandler',
    #password
    database='sound_spot'
)

# Create a cursor to execute SQL queries
cursor = connexion.cursor()

ModuleNotFoundError: No module named 'mysql'

##### En vrac

##### Analyse de sentiment

In [ ]:
%pip install textblob
%pip install nrclex

In [17]:
import nltk
nltk.download('punkt')

from textblob import TextBlob
from nrclex import NRCLex

# Texte de la chanson que vous souhaitez analyser
lyrics = """
I had a bad week
Spent the evening pretending it wasn't that deep
You could see in my eyes that it was taking over
I guess I was just blind and caught up in the moment
You know you take all of my stress right down
Help me get it off my chest and out
Into the ether with the rest of this mess that just keeps us depressed
We forget that we're here right now
'Cause we're living life at a different pace, stuck in a constant race
Keep the pressure on, you're bound to break
Something's got to change
We should just be cancelling all our plans, and not give a damn
If we're missing out on what the people think is right
Seeing through a picture behind the screen and forget to be
Lose the conversation for the message that you'll never read
I think maybe you and me
Oh, we should head out to the place where the music plays
And then we'll go all night
Two-stepping with the woman I love
All my troubles turn to nothing when I'm in your eyes
Electrified, we'll keep turning up and go all night
Oh, we had dips and falls in our time
But we know what it feels to be low then up, alone in love
And all we need is us to go all
Night, night
Two-stepping with the woman I love
Night, yeah
All we need is us
What do you reckon, is it just me?
Words are weapons and occasionally they cut deep
Crisis of confidence, it tends to come when I feel the dark
And I open my heart
If you don't see it, you should trust me
I feel like I got nothing left right now
Except this beauty in a dress right now
She got me feeling like the best, and the rest are just less
Than she needs, so we press play and step to the beat
'Cause we're living life at a different pace, stuck in a constant race
Keep the pressure on, you're bound to break
Something's got to change
We should just be cancelling all our plans, and not give a damn
Head out to the place where it plays
And we'll go all night
Two-stepping with the woman I love
All my troubles turn to nothing when I'm in your eyes
Electrified, we'll keep turning up and go all night
Oh, we had dips and falls in our time
But we know what it feels to be low then up, alone in love
And all we need is us to go all
Night, night
Two-stepping with the woman I love
Night, yeah
All we need is us to go all night
Night, night
Two-stepping with the woman I love
Night, yeah
All we need is us to go all night
"""

# Analyse du texte avec TextBlob
blob = TextBlob(lyrics)

# Analyse des émotions avec NRCLex
emotions = NRCLex(lyrics)

# Obtenez des émotions spécifiques
emotion_scores = emotions.affect_frequencies

# Affichez les scores d'émotion
for emotion, score in emotion_scores.items():
    print(f"{emotion}: {score}")

# Exemple d'accès à des émotions spécifiques
bonheur = emotion_scores["joy"]
tristesse = emotion_scores["sadness"]
colère = emotion_scores["anger"]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


fear: 0.08974358974358974
anger: 0.07692307692307693
anticip: 0.0
trust: 0.0641025641025641
surprise: 0.05128205128205128
positive: 0.19230769230769232
negative: 0.20512820512820512
sadness: 0.07692307692307693
disgust: 0.0641025641025641
joy: 0.14102564102564102
anticipation: 0.038461538461538464


In [ ]:
# Nom de l'album, de l'artiste
# Type de chanson, rythmique, popularité
# Lyrics --> Webscrapping

# À voir : category, category_playlists, featured_playlists
# playlist_add_items, playlist_change_details, artist_albums
# artist_related_artists, artist_top_tracks, audio_analysis,
# audio_features, playlist_cover_image, playlist_items
# recommendation_genre_seeds, recommandations, search,
# seek_track, user_playlists, get_access_token...

# Possibilité d'écouter de l'audio depuis le site

# Filtre pour ranger dans un certain ordre



# ................................................................
# # Retrieve the user's top tracks
# top_tracks = sp.current_user_top_tracks(limit=10, time_range='medium_term')  # You can adjust the limit and time_range

# # Print the top tracks
# for idx, track in enumerate(top_tracks['items'], start=1):
#     print(f"{idx}. {track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")

In [ ]:
sp.artists('https://open.spotify.com/intl-fr/track/7IBSffWIu7P2MC7kMwy2FM?si=b0c36190df6d467b')

In [ ]:
track['track']['artists']['id']

In [ ]:
results = sp.playlist_tracks(playlist_id)
print(results)

In [ ]:
print(playlist)

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])